# Requirements

In [103]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import re
import pickle
from collections import defaultdict
import requests

In [175]:
# load LAGT v 3.0, as we will use it as a source for metadata
LAGT = pd.read_parquet("../data/large_files/LAGT_v3-0.parquet")

In [176]:
LAGT.head(5)

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences,lemmata_source,tlg_date,not_before,not_after,date_uncertain,tlg_epithet,provenience,lemmatacount
2,ogl0001,ogl0001.ogl001,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...",180,1Kgr,"[[Πινυτός, ἀντιγράφω, θαυμάζω, ἀποδέχω, Διονύσ...",grecy,None,101.0,200.0,None,[],christian,34
8,tlg0005,tlg0005.tlg003,tlg0005.tlg003.1st1K-grc1.xml,Theocritus,Syrinx,Οὐδενὸς εὐνάτειρα Μακροπτολέμοιο δὲ μάτηρ μαί...,77,1Kgr,"[[οὐδενός, εὐνητήρ], [μακροπτολέμοιο, μήτηρ, μ...",grecy,4-3 B.C.,-400.0,-201.0,False,[Bucolici],pagan,61
9,tlg0006,tlg0006.tlg020,tlg0006.tlg020.1st1K-grc1.xml,Euripides,Fragmenta,ποίαν σε φῶμεν γαῖαν ἐκλελοιπότα πόλει ξενοῦσθ...,17708,1Kgr,"[[φημί, γῆ, ἐκλείπω, πόλις, ξενοῦσθαι], [πάτρα...",grecy,5 B.C.,-500.0,-401.0,False,[Tragici],pagan,10277
10,tlg0007,tlg0007.tlg146,tlg0007.tlg146.1st1K-grc1.xml,Plutarch,Παροιμίαι αἷς Ἀλεξανδρεῖς ἐχρῶντο,Οἴκοι τὰ Μιλήσια: ἐπὶ τῶν ὅποι μὴ προςήκει τὴν...,2685,1Kgr,"[[Μιλήσιος], [προςήκω, τρυφή, ἐπιδείκνυμι], [Ἀ...",grecy,A.D. 1-2,1.0,200.0,False,"[Biographi, Philosophici/-ae]",pagan,1488
11,tlg0007,tlg0007.tlg147,tlg0007.tlg147.1st1K-grc1.xml,Plutarch,Ἐκλογὴ περὶ τῶν ἀδυνάτων,Κατὰ πετρῶν σπείρεις. Πλίνθον πλύνεις. Δικτύῳ ...,143,1Kgr,"[[πέτρα, σπείρω], [Πλίνθος, πλύνω, Δίκτυον, ἄν...",grecy,A.D. 1-2,1.0,200.0,False,"[Biographi, Philosophici/-ae]",pagan,125


In [177]:
# currently from Glaux:
sum(LAGT["lemmata_source"] == "glaux")

835

### Load newest local Glaux

In [80]:
source_dir = "../../glaux/xml/"
glaux_filenames = os.listdir(source_dir)
glaux_filenames[:10]

['0057-094.xml',
 '0632-004.xml',
 '2042-010.xml',
 '0658-002.xml',
 '0010-009.xml',
 '1342-002.xml',
 '0057-057.xml',
 '0010-021.xml',
 '0057-043.xml',
 '0007-049.xml']

In [81]:
glaux_tlgs = [re.sub(r'(\d{4})-(\d{3}).xml', r'tlg\1.tlg\2', fn) for fn in glaux_filenames]
glaux_tlgs[:10]

['tlg0057.tlg094',
 'tlg0632.tlg004',
 'tlg2042.tlg010',
 'tlg0658.tlg002',
 'tlg0010.tlg009',
 'tlg1342.tlg002',
 'tlg0057.tlg057',
 'tlg0010.tlg021',
 'tlg0057.tlg043',
 'tlg0007.tlg049']

In [156]:
LAGT["doc_id"].isin(glaux_tlgs).sum()

1173

In [157]:
not_lagt = [glaux_fn for glaux_fn in glaux_filenames if re.sub(r'(\d{4})-(\d{3}).xml', r'tlg\1.tlg\2', glaux_fn) not in list(LAGT["doc_id"])]
not_lagt[:10]

['0632-004.xml',
 '0658-002.xml',
 '1342-002.xml',
 '0336-003.xml',
 '1772-001.xml',
 '1223-001.xml',
 '0007-082a.xml',
 '0632-002a.xml',
 '0005-005.xml',
 '0261-003.xml']

In [158]:
len(not_lagt)

248

In [159]:
with open(source_dir  + not_lagt[0], "r") as f:
    soup = BeautifulSoup(f.read())

In [160]:
print(soup.prettify())

<treebank version="2" xml:lang="grc">
 <sentence analysis="auto" document_id="0632-004" id="1" struct_id="178136">
  <word div_page="114" form="εἰ" head="102947308" id="102947284" lemma="εἰ" postag="c--------" relation="AuxC">
  </word>
  <word div_page="114" form="μὲν" head="102947308" id="102947285" lemma="μέν" postag="g--------" relation="AuxY">
  </word>
  <word div_page="114" form="εὑρίσκεις" head="102947284" id="102947286" lemma="εὑρίσκω" postag="v2spia---" relation="ADV">
  </word>
  <word div_page="114" form="τοὺς" head="102947288" id="102947287" lemma="ὁ" postag="l-p---ma-" relation="ATR">
  </word>
  <word div_page="114" form="ἀστέρας" head="102947286" id="102947288" lemma="ἀστήρ" postag="n-p---ma-" relation="OBJ">
  </word>
  <word div_page="114" form="τοὺς" head="102947290" id="102947289" lemma="ὁ" postag="l-p---ma-" relation="ATR">
  </word>
  <word div_page="114" form="ἔχοντας" head="102947288" id="102947290" lemma="ἔχω" postag="v-pppama-" relation="AT

### Extract Glaux sentence level data

In [6]:
target_path = "../data/large_files/sents_data/"
try:
    os.mkdir(target_path)
except:
    pass

In [13]:
def from_filename_to_sentence_data(fn):
    with open(source_dir    + fn, "r") as f:
        soup = BeautifulSoup(f.read())
    sentences_data = []
    for sent_n, sent in enumerate(soup.find_all("sentence")):
        sentence = ""
        sent_data = []
        start_index = 0
        for n, w in enumerate(sent.find_all("word")):
            token = w["form"]
            try:
                if (w["relation"] in ["AuxX", "AuxK", "PUNCT"]) or (n==0):
                    sentence += token
                else:
                    sentence += " " + token
                    start_index += 1
            except:
                sentence += " " + token
                start_index += 1
            end_index = start_index + len(token)
            word_index = (start_index, end_index)
            start_index = end_index
            try:
                sent_data.append((token, w["lemma"], w["postag"][0], word_index))
            except:
                sent_data.append((token, token, None, word_index))
        sentences_data.append((fn.partition(".")[0], sent_n, sentence, sent_data))
    target_fn = re.sub(r'(\d{4})-(\d{3}).xml', r'tlg\1.tlg\2', fn) + ".pickle"
    with open(target_path + target_fn, "wb") as f:
        pickle.dump(sentences_data, f)

In [14]:
for fn in glaux_filenames:
    from_filename_to_sentence_data(fn)

In [35]:
# ok, this approach does not work for all the files....
files_to_merge = [fn for fn in os.listdir(target_path) if "xml" in fn]
files_to_merge[:10]

['0632-002a.xml.pickle',
 '0541-042b.xml.pickle',
 '0627-024a.xml.pickle',
 '0007-051b.xml.pickle',
 '0007-082b.xml.pickle',
 '0632-002d.xml.pickle',
 '0632-002c.xml.pickle',
 '0093-010b.xml.pickle',
 '0096-002b.xml.pickle',
 '0007-052b.xml.pickle']

In [39]:
# let's rename and merge these files now.
file_groups = defaultdict(list)
pattern = re.compile(r'(\d{4})-(\d{3})[a-z]')

for fn in files_to_merge:
    match = pattern.match(fn)
    if match:
        doc_id = f'tlg{match.group(1)}.tlg{match.group(2)}'
        file_groups[doc_id].append(fn)

# Step 3: Iterate over groups and merge files
for doc_id, files in file_groups.items():
    files.sort()
    merged_list = []
    for fn in files:
        file_path = os.path.join(target_path, fn)
        with open(file_path, "rb") as f:
            data = pickle.load(f)
            merged_list.extend(data)

    # Step 4: Save the merged list
    target_fn = os.path.join(target_path, doc_id + ".pickle")
    with open(target_fn, "wb") as f:
        pickle.dump(merged_list, f)

In [40]:
for fn in files_to_merge:
    os.remove(os.path.join(target_path, fn))

### Add current glaux data to LAGT

In [42]:
# open a test file to explore its structure
file_path = target_path + "tlg0632.tlg002.pickle"
with open(file_path, "rb") as f:
    sentences_data = pickle.load(f)
sentences_data[:5]

[('0632-002a',
  0,
  'ὦ νέοι, ἀλλὰ σέβεσθε μεθ’ ἡσυχίας τάδε πάντα',
  [('ὦ', 'ὦ', 'i', (0, 3)),
   ('νέοι', 'νέος', 'a', (4, 9)),
   (',', ',', 'u', (9, 10)),
   ('ἀλλὰ', 'ἀλλά', 'c', (11, 17)),
   ('σέβεσθε', 'σέβω', 'v', (18, 26)),
   ('μεθ’', 'μετά', 'r', (27, 31)),
   ('ἡσυχίας', 'ἡσυχία', 'n', (32, 41)),
   ('τάδε', 'ὅδε', 'p', (42, 47)),
   ('πάντα', 'πᾶς', 'p', (48, 54))]),
 ('0632-002b',
  0,
  'εἴ τις ἐρεῖ " θεός εἰμι πάρεξ ἑνός ", οὗτος ὀφείλει κόσμον ἴσον τούτῳ στήσας εἰπεῖν " ἐμὸς οὗτος "· E',
  [('εἴ', 'εἰ', 'c', (0, 4)),
   ('τις', 'τις', 'p', (5, 8)),
   ('ἐρεῖ', 'λέγω', 'v', (9, 15)),
   ('"', '"', 'u', (16, 17)),
   ('θεός', 'θεός', 'n', (18, 23)),
   ('εἰμι', 'εἰμί', 'v', (24, 29)),
   ('πάρεξ', 'παρέξ', 'r', (30, 36)),
   ('ἑνός', 'εἷς', 'm', (37, 43)),
   ('"', '"', 'u', (44, 45)),
   (',', ',', 'u', (45, 46)),
   ('οὗτος', 'οὗτος', 'p', (47, 54)),
   ('ὀφείλει', 'ὀφείλω', 'v', (55, 

In [73]:
[sent[3] for sent in sentences_data][:3]

[[('ὦ', 'ὦ', 'i', (0, 3)),
  ('νέοι', 'νέος', 'a', (4, 9)),
  (',', ',', 'u', (9, 10)),
  ('ἀλλὰ', 'ἀλλά', 'c', (11, 17)),
  ('σέβεσθε', 'σέβω', 'v', (18, 26)),
  ('μεθ’', 'μετά', 'r', (27, 31)),
  ('ἡσυχίας', 'ἡσυχία', 'n', (32, 41)),
  ('τάδε', 'ὅδε', 'p', (42, 47)),
  ('πάντα', 'πᾶς', 'p', (48, 54))],
 [('εἴ', 'εἰ', 'c', (0, 4)),
  ('τις', 'τις', 'p', (5, 8)),
  ('ἐρεῖ', 'λέγω', 'v', (9, 15)),
  ('"', '"', 'u', (16, 17)),
  ('θεός', 'θεός', 'n', (18, 23)),
  ('εἰμι', 'εἰμί', 'v', (24, 29)),
  ('πάρεξ', 'παρέξ', 'r', (30, 36)),
  ('ἑνός', 'εἷς', 'm', (37, 43)),
  ('"', '"', 'u', (44, 45)),
  (',', ',', 'u', (45, 46)),
  ('οὗτος', 'οὗτος', 'p', (47, 54)),
  ('ὀφείλει', 'ὀφείλω', 'v', (55, 64)),
  ('κόσμον', 'κόσμος', 'n', (65, 72)),
  ('ἴσον', 'ἴσος', 'a', (73, 79)),
  ('τούτῳ', 'οὗτος', 'p', (80, 87)),
  ('στήσας', 'ἵστημι', 'v', (88, 95)),
  ('εἰπεῖν', 'λέγω', 'v', (96, 104)),
  ('"', '"', 'u', (105, 106)),
  ('ἐμὸς', 

In [75]:
lemmatized_sentences = [[t[1] for t in sent[3] if t[2] in ["n", "a", "v"]] for sent in sentences_data]
lemmatized_sentences

[['νέος', 'σέβω', 'ἡσυχία'],
 ['λέγω',
  'θεός',
  'εἰμί',
  'ὀφείλω',
  'κόσμος',
  'ἴσος',
  'ἵστημι',
  'λέγω',
  'ἐμός'],
 ['ἵστημι', 'λέγω', 'ἐμός', 'κατοικέω', 'ποιέω'],
 ['ποιέω'],
 ['Πυθαγόρας', 'Ἀναξιμένης'],
 ['λωΐων',
  'ἀγαθός',
  'εἰμί',
  'Πυθαγόρας',
  'γενεά',
  'κλέος',
  'μετανίστημι',
  'οἴχομαι',
  'Μίλητος'],
 ['κατερύκω', 'εὔκλεια', 'κατερύκω', 'Ἀναξιμένης', 'ἔοικα'],
 ['ὀνήας',
  'πόλις',
  'ἐκλείβω',
  'κόσμος',
  'αἱρέω',
  'ἐπικίνδυνος',
  'Μῆδος'],
 ['καλός', 'αἰθερολογέω', 'μελεδωνός', 'εἰμί', 'πατρίς', 'καλός'],
 ['μῦθος', 'πόλεμος', 'διαφέρω', 'Ἰταλιώτης'],
 ['Πυθαγόρας', 'Ἱέρων'],
 ['ἀσφαλής', 'ἐμός', 'βίος', 'ἡσύχιος', 'σός', 'προσπελάζω'],
 ['μέτριος', 'ἀνήρ', 'λιποδεής', 'Σικελικός', 'τράπεζα', 'προσδέω'],
 ['ἔχω', 'Πυθαγόρας', 'ἱκανός', 'ἡμέρα', 'ἔρχομαι'],
 ['θεραπεία', 'δυνάστης', 'βαρύς', 'ἐπαχθής', 'ἐθίζω'],
 ['μέγας', 'ἀσφαλής', 'αὐτά

In [136]:
def get_row_data(row):
    doc_id = row["doc_id"]
    try:
        file_path = target_path + doc_id + ".pickle"
        with open(file_path, "rb") as f:
            sentences_data = pickle.load(f)
        sentences = [sent[2] for sent in sentences_data]
        lemmatized_sentences = [[t[1] for t in sent[3] if t[2] in ["n", "a", "v"]] for sent in sentences_data]
    except:
        sentences = None
        lemmatized_sentences = None
    return sentences, lemmatized_sentences

In [178]:
%%time
result = LAGT.apply(lambda row: pd.Series(get_row_data(row)), axis=1)
LAGT["sentences"] = result[0]  # Extract sentences
LAGT["lemmatized_sentences"] = result[1] 

CPU times: user 12.5 s, sys: 6.36 s, total: 18.8 s
Wall time: 23.6 s


In [179]:
LAGT["lemmatized_sentences"].notnull().sum()

1179

### Adding additional files from glaux

In [163]:
not_lagt = [glaux_fn for glaux_fn in os.listdir(target_path) if glaux_fn.rpartition(".")[0] not in list(LAGT["doc_id"])]
not_lagt[:10]

['tlg0388.tlg004.pickle',
 'tlg1667.tlg004.pickle',
 'tlg0497.tlg005.pickle',
 'tlg1836.tlg001.pickle',
 'tlg0622.tlg001.pickle',
 'tlg0593.tlg001.pickle',
 'tlg0198.tlg001.pickle',
 'tlg0232.tlg002.pickle',
 'tlg0239.tlg003.pickle',
 'tlg0218.tlg002.pickle']

In [138]:
doc_id = "tlg0497.tlg005"
url = "http://data.perseus.org/catalog/urn:cts:greekLit:{}".format(doc_id)
soup = BeautifulSoup(requests.get(url).text)#

In [139]:
def get_dd_content(dt_text):
    dt_tag = soup.find('dt', string=dt_text)
    if dt_tag is not None:
        dd_tag = dt_tag.find_next_sibling('dd')
        if dd_tag is not None:
            return dd_tag.get_text(strip=True)  # Extract text and strip whitespace
    return None


work_title = get_dd_content("Work title:")
author = get_dd_content("Author:")

In [140]:
work_title


'Epigramma'

In [167]:
not_lagt_data = []
for fn in not_lagt:
    doc_id = fn.rpartition(".")[0]
    author_id = doc_id.partition(".")[0]
    author = ""
    title = ""
    try:
        url = "http://data.perseus.org/catalog/urn:cts:greekLit:{}".format(doc_id)
        soup = BeautifulSoup(requests.get(url).text)#
        try:
            author = get_dd_content("Author:")
            title = get_dd_content("Work title:")
        except:
            url = "http://data.perseus.org/catalog/urn:cts:greekLit:{}".format(author_id)
            soup = BeautifulSoup(requests.get(url).text)
            try:
                author = get_dd_content("Author:")
            except:
                pass
    except:
        pass
    file_path = target_path + doc_id + ".pickle"
    with open(file_path, "rb") as f:
        sentences_data = pickle.load(f)
    sentences = [sent[2] for sent in sentences_data]
    lemmatized_sentences = [[t[1] for t in sent[3] if t[2] in ["n", "a", "v"]] for sent in sentences_data]
    not_lagt_data.append({
        "author_id" : author_id,
        "doc_id" : doc_id,
        "author" : author,
        "title" : title,
        "sentences" : sentences,
        "lemmatized_sentences" : lemmatized_sentences,
        "source" : "glaux1",
        "lemmata_source" : "glaux1"}
    )

In [168]:
LAGT_glaux1 = pd.DataFrame(not_lagt_data)
len(LAGT_glaux1)

229

In [180]:
len(LAGT)

1710

In [181]:
LAGT = pd.concat([LAGT, LAGT_glaux1], ignore_index=True)

In [182]:
LAGT.sample(10)

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences,lemmata_source,tlg_date,not_before,not_after,date_uncertain,tlg_epithet,provenience,lemmatacount,sentences
1424,tlg0062,tlg0062.tlg027,tlg0062.tlg027.perseus-grc2.xml,Lucian,Περὶ Θυσιῶν,ἃ μὲν γὰρ ἐν ταῖς θυσίαις οἱ μάταιοι πράττουσ...,1862.0,perseus,"[[θυσία, μάταιος, πράσσω, ἑορτή, πρόσοδο...",glaux,A.D. 2,101.0,200.0,False,[Sophistae],pagan,899.0,[ἃ μὲν γὰρ ἐν ταῖς θυσίαις οἱ μάταιοι...
730,tlg2042,tlg2042.tlg075,tlg2042.tlg075.1st1K-grc1.xml,Origen,Expositio in Proverbia (fragmenta e catenis),(1) Παροιμία ἐστὶ λόγος ἀπόκρυφος διʼ ἑτέρου π...,18536.0,1Kgr,"[[παροιμία, εἰμί, λόγος, ἀπόκρυφος, πρό...",grecy,A.D. 2-3,101.0,300.0,False,[Theologici],christian,8515.0,[παροιμία ἐστὶ λόγος ἀπόκρυφος δι’ ἑτέ...
706,tlg2042,tlg2042.tlg021,tlg2042.tlg021.opp-grc1.xml,Origenes,In Jeremiam (Homiliae 12-20),"Ὃ προστάσσεται ὁ προφήτης λέγειν ὑπὸ θεοῦ, ὀφε...",48360.0,1Kgr,None,grecy,A.D. 2-3,101.0,300.0,False,[Theologici],christian,16135.0,None
265,tlg0097,tlg0097.tlg001,tlg0097.tlg001.1st1K-grc1.xml,Diogenianus of Heraclea,Παροιμίαι δημώδεις ἐκ τῆς Διογενιανοῦ συναγωγῆ...,Διογενιανοῦ περὶ παροιμιῶν. Τὴν παροιμίαν ὀνομ...,11267.0,1Kgr,None,grecy,A.D. 2,101.0,200.0,False,[Grammatici],pagan,6209.0,None
1730,tlg0007,tlg0007.tlg084,NaN,Plutarch,Aetia Romana et Graeca,NaN,NaN,glaux1,"[[γαμέω, ἅπτω, πῦρ, ὕδωρ, κελεύω], [στο...",glaux1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[διὰ τί τὴν γαμουμένην ἅπτεσθαι πυρὸς κ...
1212,tlg0014,tlg0014.tlg022,tlg0014.tlg022.perseus-grc2.xml,Demosthenes,κατὰ Ἀνδροτίωνος παρανόμων,"ὅπερ Εὐκτήμων, ὦ ἄνδρες δικασταί, παθὼν ὑπʼ Ἀν...",5634.0,perseus,"[[Εὐκτήμων, ἀνήρ, δικαστής, πάσχω, Ἀνδρ...",glaux,4 B.C.,-400.0,-301.0,False,[Oratores],pagan,2958.0,"[ὅπερ Εὐκτήμων, ὦ ἄνδρες δικασταί, πα..."
1492,tlg0085,tlg0085.tlg005,tlg0085.tlg005.perseus-grc2.xml,Aeschylus,Ἀγαμέμνων,Φύλαξ θεοὺς μὲν αἰτῶ τῶνδʼ ἀπαλλαγὴν πόνων φρο...,8254.0,perseus,"[[θεός, αἰτέω, ἀπαλλαγή, πόνος, φρουρά,...",agdt,6-5 B.C.,-600.0,-401.0,False,[Tragici],pagan,5243.0,[θεοὺς μὲν αἰτῶ τῶνδ’ ἀπαλλαγὴν πόνων ...
678,tlg2034,tlg2034.tlg027,tlg2034.tlg027.1st1K-grc1.xml,Porphyrius,Historia philosophiae (fragmenta),"Suidas v. Πορφύριος: ἔγραψε βιβλία πάμπλειστα,...",2293.0,1Kgr,None,grecy,A.D. 3,201.0,300.0,False,[Philosophici/-ae],pagan,929.0,None
1772,tlg1772,tlg1772.tlg001,NaN,None,None,NaN,NaN,glaux1,"[[γίγνομαι, αἰχμαλωτεύω, υἱός, Ἰσραήλ, ...",glaux1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[ἐγένετο, ἡνίκα ᾐχμαλωτεύθησαν οἱ υἱο..."
579,tlg1484,tlg1484.tlg001,tlg1484.tlg001.1st1K-grc1.xml,Anonymous,Martyrium Polycarpi,Ἡ ἐκκλησία τοῦ θεοῦ ἡ παροικοῦσα Σμύρναν τῇ ἐ...,2914.0,1Kgr,"[[ἐκκλησία, θεός, παροικέω, Σμύρνα, ἐκκλ...",grecy,p. A.D. 2,201.0,NaN,False,[],christian,1314.0,[ἡ ἐκκλησία τοῦ θεοῦ ἡ παροικοῦσα Σμύρ...


In [184]:
LAGT.to_parquet("../data/large_files/LAGT_glauxed.parquet", engine="pyarrow")